### 使用pytorch读取自己的数据集

**常用做法**

    通常的方法是将图片的路径和标签信息存储在一个 txt 中，然后从该 txt 中读取为一个list，这个 list 的每个元素就包含了图片数据的路径和标签信息。

**基本流程**

    1. 制作存储了图片的路径和标签信息的 txt
    2. 将这些信息转化为 list，该 list 每一个元素对应一个样本
    3. 通过通过 Dataset 类的 getitem 函数，读取数据和标签，并返回数据和标签

### 1. 制作图片数据的索引

In [2]:
import os

In [3]:
train_txt_path = os.path.join(".", "Data", "train.txt")
# 训练集路径
train_dir = os.path.join(".", "Data", "train")

valid_txt_path = os.path.join(".", "Data", "valid.txt")
# 验证集路径
valid_dir = os.path.join(".", "Data", "valid")


def gen_txt(txt_path, img_dir):
    f = open(txt_path, 'w')  # 打开text文件

    for root, s_dirs, _ in os.walk(img_dir, topdown=True):  # 获取 train文件下各文件夹名称  topdown为True，则优先遍历 top 目录
        for sub_dir in s_dirs:
            i_dir = os.path.join(root, sub_dir)  # 获取各类的文件夹 绝对路径
            img_list = os.listdir(i_dir)  # 获取类别文件夹下所有png图片的路径
            for i in range(len(img_list)):
                if not img_list[i].endswith('png'):  # 若不是png文件，跳过
                    continue
                label = img_list[i].split('_')[0]
                img_path = os.path.join(i_dir, img_list[i])
                line = img_path + ' ' + label + '\n'
                f.write(line)
    f.close()

In [4]:
train_txt_path

'.\\Data\\train.txt'

In [3]:
gen_txt(train_txt_path, train_dir)
gen_txt(valid_txt_path, valid_dir)

### 2. 构建 Dataset 子类

In [2]:
from PIL import Image
from torch.utils.data import Dataset


class MyDataset(Dataset):
    '''
    初始化 transform，transform 是一个 Compose 类型，里边有一个 list，list中就会定义了各种对图像进行处理的操作，可以设置减均值，除标准差，随机裁剪，旋
    转，翻转，仿射变换等操作。
    '''
    def __init__(self, txt_path, transform=None, target_transform=None):
        fh = open(txt_path, 'r')
        imgs = []
        for line in fh:
            line = line.rstrip()
            words = line.split() # 以空格分割为列表
            # 一个元素对应一个样本的路径和标签，其实就是 txt 中的一行
            imgs.append((words[0], int(words[1])))
        '''
        self.imgs 是一个 list，也就是一开始提到的 list，self.imgs 的一个元素是一个 str，包含图片路径，图片标签，这些信息是从 txt 文件中读取
        '''
        self.imgs = imgs  # 最主要就是要生成这个list， 然后DataLoader中给index，通过getitem读取图片数据
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        fn, label = self.imgs[index]
        img = Image.open(fn).convert('RGB')  # 像素值 0~255，在transfrom.totensor会除以255，使像素值变成 0~1
        
        '''
        PyTorch 的数据增强是将原始图片进行了处理，并不会生成新的一份图片，而是“覆盖”原图，当采用 randomcrop 之类的随机操作时，
        每个 epoch 输入进来的图片几乎不会是一模一样的，这达到了样本多样性的功能。
        '''
        if self.transform is not None:
            img = self.transform(img)  # 在这里做transform，转为tensor等等

        return img, label

    def __len__(self):
        return len(self.imgs)

### 3. DataSet通过DataLoder获取一个batch

In [6]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

train_txt_path = os.path.join(".", "Data", "train.txt")
# 数据预处理设置
normMean = [0.4948052, 0.48568845, 0.44682974]
normStd = [0.24580306, 0.24236229, 0.2603115]
normTransform = transforms.Normalize(normMean, normStd)
trainTransform = transforms.Compose([
    transforms.Resize(32),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    normTransform
])

# 构建MyDataset实例
train_data = MyDataset(txt_path=train_txt_path, transform=trainTransform)

# 构建DataLoder
train_loader = DataLoader(dataset=train_data, batch_size=10, shuffle=True)